## Download the images


We can use **GoogleDriveDownloader** form **google_drive_downloader** library in Python to download the shared files from the shared Google drive link: https://drive.google.com/file/d/1f7uslI-ZHidriQFZR966_aILjlkgDN76/view?usp=sharing

The file id in the above link is: **1f7uslI-ZHidriQFZR966_aILjlkgDN76**

In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1f7uslI-ZHidriQFZR966_aILjlkgDN76',
                                    dest_path='content/eye_gender_data.zip',
                                    unzip=True)

Unzipping...Done.


We have all the files from the shared Google drive link downloaded in the colab environment.

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [2]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set
from sklearn.metrics import f1_score

## Loading and preparing training data


In [3]:
labels = pd.read_csv("/content/content/eye_gender_data/Training_set.csv")   # loading the labels
file_paths = [[fname, '/content/content/eye_gender_data/train/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')

data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  data.append([new_img_array, train_data['label'][i]])

## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [4]:
data[0]

[array([[188, 188, 189, ..., 176, 175, 175],
        [189, 189, 188, ..., 174, 173, 172],
        [190, 189, 188, ..., 168, 167, 167],
        ...,
        [133, 137, 144, ..., 168, 167, 166],
        [134, 138, 145, ..., 165, 164, 163],
        [135, 139, 146, ..., 163, 162, 162]], dtype=uint8), 'male']

In [5]:
type(data[0])

list

In [6]:
input_shape = data[0][0].shape
input_shape

(100, 100)

In [7]:
train_images=[]
train_labels=[]
for example in data:
  ar = example[0]
  ar = ar/255
  ar = ar.reshape((image_size,image_size,1))
  train_images.append(ar)
  label = example[1]
  if label=='male':
    label = 1
  else: 
    label = 0
  train_labels.append(label)

In [8]:
train_images[0]

array([[[0.7372549 ],
        [0.7372549 ],
        [0.74117647],
        ...,
        [0.69019608],
        [0.68627451],
        [0.68627451]],

       [[0.74117647],
        [0.74117647],
        [0.7372549 ],
        ...,
        [0.68235294],
        [0.67843137],
        [0.6745098 ]],

       [[0.74509804],
        [0.74117647],
        [0.7372549 ],
        ...,
        [0.65882353],
        [0.65490196],
        [0.65490196]],

       ...,

       [[0.52156863],
        [0.5372549 ],
        [0.56470588],
        ...,
        [0.65882353],
        [0.65490196],
        [0.65098039]],

       [[0.5254902 ],
        [0.54117647],
        [0.56862745],
        ...,
        [0.64705882],
        [0.64313725],
        [0.63921569]],

       [[0.52941176],
        [0.54509804],
        [0.57254902],
        ...,
        [0.63921569],
        [0.63529412],
        [0.63529412]]])

In [9]:
train_images = np.array(train_images)

In [10]:
train_images.shape

(9220, 100, 100, 1)

In [11]:
train_images[0]

array([[[0.7372549 ],
        [0.7372549 ],
        [0.74117647],
        ...,
        [0.69019608],
        [0.68627451],
        [0.68627451]],

       [[0.74117647],
        [0.74117647],
        [0.7372549 ],
        ...,
        [0.68235294],
        [0.67843137],
        [0.6745098 ]],

       [[0.74509804],
        [0.74117647],
        [0.7372549 ],
        ...,
        [0.65882353],
        [0.65490196],
        [0.65490196]],

       ...,

       [[0.52156863],
        [0.5372549 ],
        [0.56470588],
        ...,
        [0.65882353],
        [0.65490196],
        [0.65098039]],

       [[0.5254902 ],
        [0.54117647],
        [0.56862745],
        ...,
        [0.64705882],
        [0.64313725],
        [0.63921569]],

       [[0.52941176],
        [0.54509804],
        [0.57254902],
        ...,
        [0.63921569],
        [0.63529412],
        [0.63529412]]])

In [12]:
train_labels = np.array(train_labels)
# train_labels.reshape(9220,1)

In [13]:
train_images.shape

(9220, 100, 100, 1)

In [14]:
train_images = train_images.reshape(9220,100,100)

## Building Model & Hyperparameter tuning
Now we are finally ready, and we can train the model.


In [15]:
train_images_3ch = np.stack([train_images]*3, axis=-1)
# test_images_3ch = np.stack([test_images]*3, axis=-1)

print('\nTrain_images.shape: {}, of {}'.format(train_images_3ch.shape, train_images_3ch.dtype))
# print('Test_images.shape: {}, of {}'.format(test_images_3ch.shape, test_images_3ch.dtype))


Train_images.shape: (9220, 100, 100, 3), of float64


## Resizing Image Data for Modeling

The minimum image size expected by the VGG model is 32x32 so we need to resize our images

In [16]:
import cv2

def resize_image_array(img, img_size_dims):
    img = cv2.resize(img, dsize=img_size_dims, 
                     interpolation=cv2.INTER_CUBIC)
    img = np.array(img, dtype=np.float32)
    return img

In [17]:
%%time

IMG_DIMS = (32, 32)

train_images_3ch = np.array([resize_image_array(img, img_size_dims=IMG_DIMS) for img in train_images_3ch])
# test_images_3ch = np.array([resize_image_array(img, img_size_dims=IMG_DIMS) for img in test_images_3ch])

print('\nTrain_images.shape: {}, of {}'.format(train_images_3ch.shape, train_images_3ch.dtype))
# print('Test_images.shape: {}, of {}'.format(test_images_3ch.shape, test_images_3ch.dtype))


Train_images.shape: (9220, 32, 32, 3), of float32
CPU times: user 657 ms, sys: 111 ms, total: 769 ms
Wall time: 772 ms


## Build CNN Model Architecture

We will now build our CNN model architecture customizing the VGG-19 model.

### Build Cut-VGG19 Model

In [18]:
# define input shape
INPUT_SHAPE = (32, 32, 3)

# get the VGG19 model
vgg_layers = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, 
                                               input_shape=INPUT_SHAPE)

vgg_layers.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
______________________________________________________________

In [19]:
# define sequential model
model = tf.keras.models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_layers)

# add flatten layer
model.add(tf.keras.layers.Flatten())

# add dense layers with some dropout
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.6))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.6))
model.add(tf.keras.layers.Dense(256, activation='relu'))
# model.add(tf.keras.layers.Dropout(rate=0.3))
# model.add(tf.keras.layers.Dense(128, activation='relu'))


# add output layer
model.add(tf.keras.layers.Dense(2, activation='softmax'))

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# view model layers
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1

In [20]:
EPOCHS = 100
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, 
                                               restore_best_weights=True,
                                               verbose=1)

history = model.fit(train_images_3ch, train_labels,
                    batch_size=128,
                    callbacks=[es_callback], 
                    validation_split=0.1, epochs=EPOCHS,
                    verbose=True)

Epoch 1/100
65/65 [==============================] - 44s 199ms/step - loss: 0.6714 - accuracy: 0.6135 - val_loss: 0.4569 - val_accuracy: 0.8026
Epoch 2/100
65/65 [==============================] - 10s 155ms/step - loss: 0.4124 - accuracy: 0.8208 - val_loss: 0.3396 - val_accuracy: 0.8547
Epoch 3/100
65/65 [==============================] - 10s 154ms/step - loss: 0.2780 - accuracy: 0.8883 - val_loss: 0.2738 - val_accuracy: 0.8894
Epoch 4/100
65/65 [==============================] - 10s 159ms/step - loss: 0.2141 - accuracy: 0.9164 - val_loss: 0.2456 - val_accuracy: 0.9067
Epoch 5/100
65/65 [==============================] - 10s 154ms/step - loss: 0.1785 - accuracy: 0.9312 - val_loss: 0.2632 - val_accuracy: 0.9078
Epoch 6/100
65/65 [==============================] - 10s 154ms/step - loss: 0.1449 - accuracy: 0.9443 - val_loss: 0.2407 - val_accuracy: 0.9154
Epoch 7/100
65/65 [==============================] - 10s 154ms/step - loss: 0.1292 - accuracy: 0.9513 - val_loss: 0.1955 - val_accuracy:

## Validate the model


## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

In [21]:
labels = pd.read_csv("/content/content/eye_gender_data/Testing_set.csv")   # loading the labels
file_paths = [[fname, '/content/content/eye_gender_data/test/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_data = pd.merge(images, labels, how = 'inner', on = 'filename')

data = []     # initialize an empty numpy array
image_size = 100     # image size taken is 100 here. one can take other size too
for i in range(len(test_data)):
  img_array = cv2.imread(test_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale
  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  data.append([new_img_array])

In [22]:
data[0][0].shape

(100, 100)

## Data Pre-processing on test_data


In [23]:
test_images=[]
for example in data:
  ar = example[0]
  ar = ar/255
  ar = ar.reshape((image_size,image_size,1))
  test_images.append(ar)

In [24]:
test_images[0]

array([[[0.54901961],
        [0.55294118],
        [0.55686275],
        ...,
        [0.6627451 ],
        [0.67058824],
        [0.67843137]],

       [[0.54117647],
        [0.54117647],
        [0.54117647],
        ...,
        [0.65098039],
        [0.65882353],
        [0.66666667]],

       [[0.5372549 ],
        [0.52941176],
        [0.5254902 ],
        ...,
        [0.63921569],
        [0.64313725],
        [0.65098039]],

       ...,

       [[0.57254902],
        [0.58823529],
        [0.60784314],
        ...,
        [0.59215686],
        [0.59215686],
        [0.59215686]],

       [[0.57254902],
        [0.59215686],
        [0.61568627],
        ...,
        [0.58039216],
        [0.57647059],
        [0.57254902]],

       [[0.57647059],
        [0.59607843],
        [0.61960784],
        ...,
        [0.57647059],
        [0.56862745],
        [0.56078431]]])

In [25]:
test_images = np.array(test_images)
test_images.shape

(2305, 100, 100, 1)

In [26]:
test_images = test_images.reshape(2305,100,100)

In [27]:
test_images_3ch = np.stack([test_images]*3, axis=-1)
print('Test_images.shape: {}, of {}'.format(test_images_3ch.shape, test_images_3ch.dtype))

Test_images.shape: (2305, 100, 100, 3), of float64


In [28]:
import cv2

def resize_image_array(img, img_size_dims):
    img = cv2.resize(img, dsize=img_size_dims, 
                     interpolation=cv2.INTER_CUBIC)
    img = np.array(img, dtype=np.float32)
    return img

In [29]:
%%time

IMG_DIMS = (32, 32)

test_images_3ch = np.array([resize_image_array(img, img_size_dims=IMG_DIMS) for img in test_images_3ch])

print('Test_images.shape: {}, of {}'.format(test_images_3ch.shape, test_images_3ch.dtype))

Test_images.shape: (2305, 32, 32, 3), of float32
CPU times: user 170 ms, sys: 9.93 ms, total: 180 ms
Wall time: 181 ms


### Make Prediction on Test Dataset
Time to make a submission!!!

In [30]:
predictions  = model.predict(test_images_3ch)

In [31]:
predictions[0]

array([0.00192001, 0.99808   ], dtype=float32)

In [32]:
predictions = np.array(predictions)
result = np.argmax(predictions,axis=1)

In [33]:
result[0]

1

In [34]:
len(result)

2305

In [35]:
len(test_images)

2305

In [36]:
results=[]
for value in result:
  if value:
    results.append('male')
  else:
    results.append('female')

In [37]:
results[0]

'male'

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [ ]:
# res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
# res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [38]:
res = pd.DataFrame({'label': results})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data

res.to_csv("TransferLearning_128.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('TransferLearning_128.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Well Done! 👍**
You are all set to make a submission. Let's head to the **[challenge page](https://dphi.tech/challenges/4-week-deep-learning-online-bootcamp-final-assignment-sex-determination-by-morphometry-of-eyes/144/submit)** to make the submission.